In [2]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import pickle
import time
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load my own functions
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw
from kde_training_utilities import kde_load_data
import ddm_data_simulation as ddm_sim
import boundary_functions as bf

In [3]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7747126415252714600
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14333018260300978518
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12048773940
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10136970403330521425
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:03:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16824059044018370692
physical_device_desc: "device: XLA_GPU device"
]


In [5]:
# Load Model
model_path = '/media/data_cifs/afengler/data/kde/linear_collapse/keras_models/dnnregressor_ddm_linear_collapse_06_22_19_23_27_28/model_0' 
ckpt_path = '/media/data_cifs/afengler/data/kde/linear_collapse/keras_models/dnnregressor_ddm_linear_collapse_06_22_19_23_27_28/ckpt_0_130'

model = keras.models.load_model(model_path)
model.load_weights(ckpt_path)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
dense_49 (Dense)             (None, 20)                160       
_________________________________________________________________
dense_50 (Dense)             (None, 40)                840       
_________________________________________________________________
dense_51 (Dense)             (None, 60)                2460      
_________________________________________________________________
dense_52 (Dense)             (None, 80)                4880      
_________________________________________________________________
dense_53 (Dense)             (None, 100)               8100      
_________________________________________________________________
dense_54 (Dense)             (None, 120)               12120     
__________

In [ ]:
# Initializations -----
n_runs = 300
n_samples = 2500


# NOTE PARAMETERS: WEIBULL: [v, a, w, node, shape, scale]
param_bounds = [(-2.5, 2.5), (0.5, 4), (0.15, 0.85), (0, 5), (1.01, 49), (0.01, 9.9)]


my_optim_columns = ['v_sim', 'a_sim', 'w_sim', 'node_sim', 'shape_sim', 'scale_sim',
                    'v_mle', 'a_mle', 'w_mle', 'node_mle', 'shape_mle', 'scale_mle', 'n_samples']

# Get parameter names in correct ordering:
dat = pickle.load(open('/home/afengler/git_repos/nn_likelihoods/data_storage/kde/weibull/train_test_data/test_features.pickle' , 
                       'rb'))

parameter_names = list(dat.keys())[:-2] # :-1 to get rid of 'rt' and 'choice' here

# Make columns for optimizer result table
p_sim = []
p_mle = []

for parameter_name in parameter_names:
    p_sim.append(parameter_name + '_sim')
    p_mle.append(parameter_name + '_mle')
    
my_optim_columns = p_sim + p_mle + ['n_samples']

# Initialize the data frame in which to store optimizer results
optim_results = pd.DataFrame(np.zeros((n_runs, len(my_optim_columns))), columns = my_optim_columns)
optim_results.iloc[:, 2 * len(parameter_names)] = n_samples

# define boundary
boundary = bf.weibull_bnd
boundary_multiplicative = False

# Define the likelihood function
def log_p(params = [0, 1, 0.9], model = [], data = [], parameter_names = []):
    # Make feature array
    feature_array = np.zeros((data[0].shape[0], len(parameter_names) + 2))
    
    # Store parameters
    cnt = 0
    for i in range(0, len(parameter_names), 1):
        feature_array[:, i] = params[i]
        cnt += 1
    
    # Store rts and choices
    feature_array[:, cnt] = data[0].ravel() # rts
    feature_array[:, cnt + 1] = data[1].ravel() # choices
    
    # Get model predictions
    prediction = model.predict(feature_array)
    
    # Some post-processing of predictions
    prediction[prediction < 1e-29] = 1e-29
    
    return(- np.sum(np.log(prediction)))  

def make_params(param_bounds = []):
    params = np.zeros(len(param_bounds))
    
    for i in range(len(params)):
        params[i] = np.random.uniform(low = param_bounds[i][0], high = param_bounds[i][1])
        
    return
# ---------------------

In [ ]:
# Main loop ----------- TD: Parallelize
for i in range(0, n_runs, 1): 
    
    # Get start time
    start_time = time.time()
    
#     # Sample parameters
#     v_sim = np.random.uniform(high = v_range[1], low = v_range[0])
#     a_sim = np.random.uniform(high = a_range[1], low = a_range[0])
#     w_sim = np.random.uniform(high = w_range[1], low = w_range[0])

#     #c1_sim = np.random.uniform(high = c1_range[1], low = c1_range[0])
#     #c2_sim = np.random.uniform(high = c2_range[1], low = c2_range[0])
#     node_sim = np.random.uniform(high = node_range[1], low = node_range[0])
#     shape_sim = np.random.uniform(high = shape_range[1], low = shape_range[0])
#     scale_sim = np.random.uniform(high = scale_range[1], low = scale_range[0])

    tmp_params = make_params(param_bounds = param_bounds)
    
    # Store in output file
    optim_results.iloc[i, :len(parameter_names)] = tmp_params
    
    # Print some info on run
    print('Parameters for run ' + str(i) + ': ')
    print([v_sim, a_sim, w_sim, node_sim, shape_sim, scale_sim])
    
    # Run model simulations
    ddm_dat_tmp = ddm_sim.ddm_flexbound_simulate(v = tmp_params[0],
                                                 a = tmp_params[1],
                                                 w = tmp_params[2],
                                                 s = 1,
                                                 delta_t = 0.001,
                                                 max_t = 20,
                                                 n_samples = n_samples,
                                                 boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                                 boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                                 boundary_params = {'node': tmp_params[3], 
                                                                    'shape': tmp_params[4],
                                                                    'scale': tmp_params[5]})
        
    # Print some info on run
    print('Mean rt for current run: ')
    print(np.mean(ddm_dat_tmp[0]))
    
    # Run optimizer
    out = differential_evolution(log_p, 
                                 bounds = [(v_range[0],v_range[1]), 
                                           (a_range[0], a_range[1]), 
                                           (w_range[0], w_range[1]),
                                           (node_range[0], node_range[1]),
                                           (shape_range[0], shape_range[1]),
                                           (scale_range[0], scale_range[1])], 
                                 args = (model, ddm_dat_tmp, parameter_names), 
                                 popsize = 30,
                                 disp = True)
    
    # Print some info
    print('Solution vector of current run: ')
    print(out.x)
    
    print('The run took: ')
    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
    # Store result in output file
    optim_results.iloc[i, len(parameter_names):(2*len(parameter_names))] = out.x
# -----------------------

# Save optimization results to file
optim_results.to_csv(os.getcwd() + '/experiments/kde_ddm_weibull_mle/optim_results.csv')

In [ ]:
# Read in results
optim_results = pd.read_csv(os.getcwd() + '/experiments/ddm_flexbound_kde_mle_fix_v_0_c1_0_w_unbiased_arange_2_3/optim_results.csv')

In [ ]:
plt.scatter(optim_results['v_sim'], optim_results['v_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for v
reg = LinearRegression().fit(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))
reg.score(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))

In [ ]:
plt.scatter(optim_results['a_sim'], optim_results['a_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for a
reg = LinearRegression().fit(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))
reg.score(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))

In [ ]:
plt.scatter(optim_results['w_sim'], optim_results['w_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))
reg.score(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))

In [ ]:
plt.scatter(optim_results['c1_sim'], optim_results['c1_mle'])

In [ ]:
# Regression for c1
reg = LinearRegression().fit(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))
reg.score(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))

In [ ]:
plt.scatter(optim_results['c2_sim'], optim_results['c2_mle'], c = optim_results['a_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))
reg.score(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))